In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.neural_network import MLPRegressor, MLPClassifier

from sklearn.metrics import mean_squared_error, f1_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

2022-04-02 17:52:57.743770: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-02 17:52:57.743819: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Инициализируем все известные генераторы случайных чисел / Setting all known random seeds
my_code = "БевзА"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

tf.compat.v1.set_random_seed(my_seed)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

2022-04-02 17:52:59.688315: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-02 17:52:59.688887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-02 17:52:59.688951: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-02 17:52:59.689008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-mosalov-2dnotebook-5ffor-5fai-5fmain-2diu8oknl3): /proc/driver/nvidia/version does not exist


In [3]:
# Читаем данные из файла
data_B = pd.read_csv("datasets/Fish.csv")

In [4]:
data_B

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...,...
154,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data_B))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data_B, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
ord_columns = ['Species']

ct = ColumnTransformer(transformers=[
    ('numerical', MinMaxScaler(), num_columns), 
    ('ordinal', OneHotEncoder(), ord_columns)])

ct.fit(train)

ColumnTransformer(transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width']),
                                ('ordinal', OneHotEncoder(), ['Species'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = num_columns + list(range(7))
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [10]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,0,1,2,3,4,5,6
0,0.06250,0.179381,0.186047,0.188406,0.205218,0.310741,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.12500,0.301031,0.292636,0.326087,0.332930,0.425932,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.18750,0.498969,0.496124,0.525362,0.233728,0.430336,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.17000,0.360825,0.360465,0.394928,0.401720,0.518360,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.18750,0.400000,0.393411,0.385870,0.343996,0.498599,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.06875,0.237113,0.244186,0.248188,0.232829,0.344115,0.0,0.0,1.0,0.0,0.0,0.0,0.0
91,0.11250,0.319588,0.321705,0.320652,0.276704,0.362489,0.0,0.0,1.0,0.0,0.0,0.0,0.0
92,0.08750,0.237113,0.238372,0.260870,0.399934,0.306852,0.0,1.0,0.0,0.0,0.0,0.0,0.0
93,0.43750,0.472165,0.476744,0.538043,0.775858,0.578972,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# В качестве входных параметров используем все числовые параметры
# в качестве выходного - единственный категориальный параметр.
x_labels = num_columns[:]
y_labels = column_names[6:]
print(x_labels)
print(y_labels)

['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
[0, 1, 2, 3, 4, 5, 6]


In [12]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]


In [13]:
y_train

,0,1,2,3,4,5,6
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
90,0.0,0.0,1.0,0.0,0.0,0.0,0.0
91,0.0,0.0,1.0,0.0,0.0,0.0,0.0
92,0.0,1.0,0.0,0.0,0.0,0.0,0.0
93,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Задание №1 - решение задачи классификации.
# В качестве входных параметров используем все числовые параметры,
# в качестве выходного - единственный категориальный параметр.

In [15]:
# Создайте нейронную сеть для решения задачи классификации двумя способами: с помощью sklearn и keras.
# Сравните их эффективность.
# Для keras используйте loss и metrics = 'categorical_crossentropy'.

In [16]:
# Создадим нейроннную сеть для решения задачи классификации на базе библиотеки sklearn
# Выбираем количество итераций 10000 для оптимизации
reg = MLPClassifier(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=10000)
reg.get_params()

{'activation': 'relu',
 'alpha': 0.0,
 'batch_size': 16,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-07,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 10000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [17]:
# Обучим нейронную сеть
reg.fit(x_train, y_train)

MLPClassifier(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=10000)

In [18]:
pred_val1 = reg.predict(x_val)


In [19]:
# Проверим работу обученной нейронной сети на валидационной выборке

mse1 = mean_squared_error(y_val, pred_val1)
print(mse1)

0.022321428571428572


In [20]:
# Создадим нейронную сеть на базе библиотеки keras
model = Sequential()

model.add(Dense(7, input_dim=6, activation='sigmoid', use_bias=False))

model.add(Dense(7,input_dim=6, activation='sigmoid', use_bias=False))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 42        
                                                                 
 dense_1 (Dense)             (None, 7)                 49        
                                                                 
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Обучим нейронную сеть
history1 = model.fit(x_train, y_train
                    , validation_data = (x_val, y_val)
                    , epochs=15, batch_size=6, verbose=1)

Epoch 1/15
16/16 [==============================] - 1s 18ms/step - loss: 2.1588 - categorical_crossentropy: 2.1588 - val_loss: 2.3349 - val_categorical_crossentropy: 2.3349
Epoch 2/15
16/16 [==============================] - 0s 5ms/step - loss: 2.1236 - categorical_crossentropy: 2.1236 - val_loss: 2.2811 - val_categorical_crossentropy: 2.2811
Epoch 3/15
16/16 [==============================] - 0s 6ms/step - loss: 2.0886 - categorical_crossentropy: 2.0886 - val_loss: 2.2343 - val_categorical_crossentropy: 2.2343
Epoch 4/15
16/16 [==============================] - 0s 7ms/step - loss: 2.0588 - categorical_crossentropy: 2.0588 - val_loss: 2.1870 - val_categorical_crossentropy: 2.1870
Epoch 5/15
16/16 [==============================] - 0s 6ms/step - loss: 2.0299 - categorical_crossentropy: 2.0299 - val_loss: 2.1455 - val_categorical_crossentropy: 2.1455
Epoch 6/15
16/16 [==============================] - 0s 7ms/step - loss: 2.0048 - categorical_crossentropy: 2.0048 - val_loss: 2.1029 - val_

In [22]:
pred_val2 = model.predict(x_val)
mse2 = mean_squared_error(y_val, pred_val2)
print(mse2)

0.3092290168267275


In [23]:
# Задание №2 - использование dropout-слоя.

In [24]:
# Создайте нейронную сеть для решения задачи классификации с помощью keras. Используйте dropout-слой.
# Сравните эффективность нейронных сетей с dropout-слоем и без него.
# Попробуйте найти такие параметры dropout-слоя, чтобы сеть с ним работала лучше, чем без него.

In [25]:
drop = Sequential()

drop.add(Dense(100, input_dim=6, activation='relu', use_bias=False))

drop.add(Dropout(rate=0.5))

drop.add(Dense(7, activation='linear', use_bias=False))

drop.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

drop.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               600       
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 700       
                                                                 
Total params: 1,300
Trainable params: 1,300
Non-trainable params: 0
_________________________________________________________________


In [26]:
history2 = drop.fit(x_train, y_train, validation_data = (x_val, y_val), 
                    epochs=30, batch_size=5, verbose=1)

Epoch 1/30
19/19 [==============================] - 1s 11ms/step - loss: 0.1664 - mean_squared_error: 0.1664 - val_loss: 0.1201 - val_mean_squared_error: 0.1201
Epoch 2/30
19/19 [==============================] - 0s 3ms/step - loss: 0.1437 - mean_squared_error: 0.1437 - val_loss: 0.1084 - val_mean_squared_error: 0.1084
Epoch 3/30
19/19 [==============================] - 0s 6ms/step - loss: 0.1271 - mean_squared_error: 0.1271 - val_loss: 0.1057 - val_mean_squared_error: 0.1057
Epoch 4/30
19/19 [==============================] - 0s 5ms/step - loss: 0.1273 - mean_squared_error: 0.1273 - val_loss: 0.1033 - val_mean_squared_error: 0.1033
Epoch 5/30
19/19 [==============================] - 0s 4ms/step - loss: 0.1225 - mean_squared_error: 0.1225 - val_loss: 0.1022 - val_mean_squared_error: 0.1022
Epoch 6/30
19/19 [==============================] - 0s 3ms/step - loss: 0.1218 - mean_squared_error: 0.1218 - val_loss: 0.1007 - val_mean_squared_error: 0.1007
Epoch 7/30
19/19 [=====================

In [27]:
pred_val3 = drop.predict(x_val)
mse3 = mean_squared_error(y_val, pred_val3)
print(mse3)

0.07033518273280183


In [28]:
print(mse1)
print(mse2)
print(mse3)

0.022321428571428572
0.3092290168267275
0.07033518273280183
